# **Keboola script to get the data from APIs to .csv**

In [20]:
from pprint import pprint
import pandas as pd
#from pandas.io.json import json_normalize -- keboola, sandbox
import json
import requests
import csv
import os

In [21]:
trip_id = ['331_241_200302', '331_260_200302', '331_256_200302', '331_242_200302', '331_257_200302', '331_243_200302', 
           '332_691_200316', '332_680_200316', '332_415_190902', '332_116_200316', '332_696_200316', '332_422_200316', 
           '332_59_190831', '332_102_200316', '332_692_200316', '332_412_190902', '332_108_200316', '332_33_190831', 
           '332_88_200316', '333_1119_200302', '333_557_200302', '333_1064_200302', '333_1051_200302', '333_1115_200302', 
           '333_1093_200302', '333_1012_200302', '333_558_200302', '333_1052_200302', '333_1071_200302', '333_1094_200302', 
           '333_602_200302', '333_715_200302', '333_751_200302', '333_517_200302', '333_560_200302', '333_1072_200302', 
           '333_677_200302', '333_604_200302', '335_351_191111', '335_360_200316', '335_358_191111', '335_339_191111', 
           '335_355_200316','332_27_200316', '332_698_200316', '332_417_191101', '332_682_200316', '332_687_200316', 
           '332_694_200316', '332_19_200316', '332_428_191101', '332_673_200316', '332_688_200316', '332_683_200316', 
           '332_702_200316', '335_21_200316', '335_29_191111', '335_347_200316', '335_11_200316', '335_349_191111', 
           '335_345_200316', '335_42_200316', '335_353_191111', '331_252_200302', '331_258_200302', '331_248_200302', 
           '331_255_200302', '331_262_200302', '333_1110_200302', '333_1117_200302', '333_1078_200302', '333_1068_200302', 
           '333_1087_200302', '333_1100_200302', '333_1057_200302', '333_1111_200302', '333_1069_200302', '333_1079_200302', 
           '333_1058_200302', '333_1101_200302']
#print(len(trip_id))

In [22]:
file_key = open('api_key.txt', encoding='utf-8')

In [23]:
key = [key.strip() for key in file_key]

In [24]:
for tripid in trip_id:
    api_key = key[0]
    headers = {
    'Content-Type': 'application/json; charset=utf-8',
    'x-access-token': api_key
  }
    response1 = requests.get(f'https://api.golemio.cz/v2/vehiclepositions/{tripid}?includePositions=true', headers = headers)
    #print(response1)

In [26]:
if response1.status_code == 200:
    trip_id_info_all = json.loads(response1.text) 
    
    coordinates = [ trip_id_info_all['properties']['all_positions']['features'][x]['geometry']['coordinates'] for x in range(len(trip_id_info_all['properties']['all_positions']['features']))]
    longitude = [gps[0] for gps in coordinates]
    latitude = [gps[1] for gps in coordinates]
    
    final = json_normalize(trip_id_info_all['properties']['all_positions']['features'])
    
    final['trip_id'] = tripid
    final['latitude'] = latitude
    final['longitude'] = longitude
    
    newCols = ['type',
               'geometry_coordinates',
               'geometry_type',
               'properties_bearing',
               'properties_delay_actual',
               'properties_delay_last_stop_arrival',
               'properties_delay_last_stop_departure',
               'properties_is_canceled',
               'properties_last_stop_arrival_time',
               'properties_last_stop_departure_time',
               'properties_last_stop_id',
               'properties_last_stop_sequence',
               'properties_next_stop_arrival_time',
               'properties_next_stop_departure_time',
               'properties_next_stop_id',
               'properties_next_stop_sequence',
               'properties_origin_timestamp',
               'properties_shape_dist_traveled',
               'properties_speed',
               'trip_id',
               'latitude',
               'longitude']
    final.columns = newCols  
    if not os.path.isfile('/data/out/tables/all_positions.csv'): 
        final.to_csv('/data/out/tables/all_positions.csv', mode = 'a', sep=',', encoding="utf-8", index=False)
    else:
        final.to_csv('/data/out/tables/all_positions.csv', mode = 'a', sep=',', encoding="utf-8", index=False, header=False)
else:
    final = pd.DataFrame(columns=['type',
           'geometry_coordinates',
           'geometry_type',
           'properties_bearing',
           'properties_delay_actual',
           'properties_delay_last_stop_arrival',
           'properties_delay_last_stop_departure',
           'properties_is_canceled',
           'properties_last_stop_arrival_time',
           'properties_last_stop_departure_time',
           'properties_last_stop_id',
           'properties_last_stop_sequence',
           'properties_next_stop_arrival_time',
           'properties_next_stop_departure_time',
           'properties_next_stop_id',
           'properties_next_stop_sequence',
           'properties_origin_timestamp',
           'properties_shape_dist_traveled',
           'properties_speed',
           'trip_id',
           'latitude',
           'longitude'])
    #keboola -> /data/out/tables/all_positions.csv 
    if not os.path.isfile('all_positions.csv'): 
        final.to_csv('all_positions.csv', mode = 'a', sep=',', encoding="utf-8", index=False)
    else:
        final.to_csv('all_positions.csv', mode = 'a', sep=',', encoding="utf-8", index=False, header=False)